In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 데이터 로드

In [2]:
# Load datasets
esg_file_path = 'ESG.csv'
finance_file_path = 'finance.csv'
marketcap_file_path = 'marketcap.csv'

esg_data = pd.read_csv(esg_file_path, encoding="cp949", low_memory=False)
finance_data = pd.read_csv(finance_file_path, encoding="cp949", low_memory=False)
marketcap_data = pd.read_csv(marketcap_file_path, encoding="cp949", low_memory=False)

In [3]:
esg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10990 entries, 0 to 10989
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NO         10989 non-null  float64
 1   기업명        10989 non-null  object 
 2   기업코드       10989 non-null  float64
 3   ESG등급      10989 non-null  object 
 4   환경         10989 non-null  object 
 5   사회         10989 non-null  object 
 6   지배구조       10989 non-null  object 
 7   평가년도       10989 non-null  float64
 8   ESG 등급조정   165 non-null    object 
 9   환경 등급조정    57 non-null     object 
 10  사회 등급조정    274 non-null    object 
 11  지배구조 등급조정  213 non-null    object 
dtypes: float64(3), object(9)
memory usage: 1.0+ MB


In [4]:
finance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33012 entries, 0 to 33011
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        33012 non-null  int64  
 1   Symbol            33012 non-null  object 
 2   Ticker            32949 non-null  object 
 3   Name              33012 non-null  object 
 4   종목명 (Full)        32949 non-null  object 
 5   기업명 (한글)          32949 non-null  object 
 6   거래소 업종            31179 non-null  object 
 7   거래소 업종코드          31179 non-null  object 
 8   결산월               33012 non-null  int64  
 9   회계년               33012 non-null  int64  
 10  주기                33012 non-null  object 
 11  자기자본비율(%)         28597 non-null  object 
 12  차입금의존도(%)         28597 non-null  float64
 13  총부채(천원)           28600 non-null  float64
 14  영업이익률(%)          28321 non-null  float64
 15  매출액(천원)           28600 non-null  float64
 16  총자본회전율(회)         27771 non-null  object

In [5]:
marketcap_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38542 entries, 0 to 38541
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       38542 non-null  int64  
 1   Ticker           38471 non-null  object 
 2   회계년              38542 non-null  int64  
 3   시가총액(백만원)        29571 non-null  float64
 4   WACC(시가총액평균)(%)  27171 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ MB


# ESG 전처리

In [6]:
# 1 하나라도 미공개나 등급없음을 다 지우는 방법 -> data 너무 적을 것 같음 # 정우

filtered_data = esg_data[(esg_data['ESG등급'] != '미공개') & (esg_data['환경'] != '미공개') &
                         (esg_data['사회'] != '미공개') & (esg_data['지배구조'] != '미공개')]

filtered_data = filtered_data[(esg_data['ESG등급'] != '등급없음') & (esg_data['환경'] != '등급없음') &
                         (esg_data['사회'] != '등급없음') & (esg_data['지배구조'] != '등급없음')]

filtered_data = filtered_data.dropna(subset=['ESG등급'])
filtered_data = filtered_data.dropna(subset=['환경'])
filtered_data = filtered_data.dropna(subset=['사회'])
filtered_data = filtered_data.dropna(subset=['지배구조'])

#Exclude 2024
filtered_data = filtered_data[filtered_data['평가년도'] != 2024.]
filtered_data = filtered_data[filtered_data['평가년도'] > 2019.]

filtered_data.head(3)

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\803686864.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(esg_data['ESG등급'] != '등급없음') & (esg_data['환경'] != '등급없음') &


,NO,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도,ESG 등급조정,환경 등급조정,사회 등급조정,지배구조 등급조정
4,10986.0,AJ네트웍스,95570.0,B,D,B,B,2020.0,NaN,NaN,NaN,NaN
5,10985.0,AJ네트웍스,95570.0,B,D,B,B,2021.0,NaN,NaN,NaN,NaN
6,10984.0,AJ네트웍스,95570.0,B+,C,B+,B+,2022.0,NaN,NaN,NaN,NaN


In [7]:
# 1 하나라도 미공개나 등급없음을 다 지우는 방법 -> data 너무 적을 것 같음 # 정우
# shift

filtered_data = esg_data[(esg_data['ESG등급'] != '미공개') & (esg_data['환경'] != '미공개') &
                         (esg_data['사회'] != '미공개') & (esg_data['지배구조'] != '미공개')]

filtered_data = filtered_data[(esg_data['ESG등급'] != '등급없음') & (esg_data['환경'] != '등급없음') &
                         (esg_data['사회'] != '등급없음') & (esg_data['지배구조'] != '등급없음')]

filtered_data = filtered_data.dropna(subset=['ESG등급'])
filtered_data = filtered_data.dropna(subset=['환경'])
filtered_data = filtered_data.dropna(subset=['사회'])
filtered_data = filtered_data.dropna(subset=['지배구조'])

#Exclude 2024
#filtered_data = filtered_data[filtered_data['평가년도'] != 2024.]
filtered_data = filtered_data[filtered_data['평가년도'] > 2020.]

filtered_data.head(3)

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\1534352060.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data = filtered_data[(esg_data['ESG등급'] != '등급없음') & (esg_data['환경'] != '등급없음') &


,NO,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도,ESG 등급조정,환경 등급조정,사회 등급조정,지배구조 등급조정
5,10985.0,AJ네트웍스,95570.0,B,D,B,B,2021.0,NaN,NaN,NaN,NaN
6,10984.0,AJ네트웍스,95570.0,B+,C,B+,B+,2022.0,NaN,NaN,NaN,NaN
7,10983.0,AJ네트웍스,95570.0,B+,C,A,B+,2023.0,NaN,NaN,NaN,NaN


In [8]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3912 entries, 5 to 10989
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NO         3912 non-null   float64
 1   기업명        3912 non-null   object 
 2   기업코드       3912 non-null   float64
 3   ESG등급      3912 non-null   object 
 4   환경         3912 non-null   object 
 5   사회         3912 non-null   object 
 6   지배구조       3912 non-null   object 
 7   평가년도       3912 non-null   float64
 8   ESG 등급조정   44 non-null     object 
 9   환경 등급조정    11 non-null     object 
 10  사회 등급조정    74 non-null     object 
 11  지배구조 등급조정  63 non-null     object 
dtypes: float64(3), object(9)
memory usage: 397.3+ KB


In [9]:
filtered_data_year = filtered_data.groupby('기업코드').filter(lambda x: x['평가년도'].nunique() > 3)
filtered_data_year['기업코드'].nunique()

877

In [10]:
finance_data['기업코드'] = finance_data['Ticker'].str.extract('(\d+)', expand=False)  # Extract numeric part of the ticker
finance_data['기업코드'] = finance_data['기업코드'].astype(float)
finance_data['평가년도'] = finance_data['회계년'].astype(float)

filtered_data_year['기업코드'] = filtered_data_year['기업코드'].astype(float)
filtered_data_year['평가년도'] = filtered_data_year['평가년도'].astype(float)

marketcap_data['기업코드'] = marketcap_data['Ticker'].str.extract('(\d+)', expand=False)  # Extract numeric part of the ticker
marketcap_data['기업코드'] = marketcap_data['기업코드'].astype(float)
marketcap_data['평가년도'] = marketcap_data['회계년'].astype(float)

In [11]:
# shift
filtered_data_year['평가년도'] = filtered_data_year['평가년도'] - 1
filtered_data_year

,NO,기업명,기업코드,ESG등급,환경,사회,지배구조,평가년도,ESG 등급조정,환경 등급조정,사회 등급조정,지배구조 등급조정
5,10985.0,AJ네트웍스,95570.0,B,D,B,B,2020.0,NaN,NaN,NaN,NaN
6,10984.0,AJ네트웍스,95570.0,B+,C,B+,B+,2021.0,NaN,NaN,NaN,NaN
7,10983.0,AJ네트웍스,95570.0,B+,C,A,B+,2022.0,NaN,NaN,NaN,NaN
8,10982.0,AJ네트웍스,95570.0,B+,C,A,B+,2023.0,NaN,NaN,NaN,NaN
24,10966.0,AK홀딩스,6840.0,B+,B,B+,B+,2020.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
10976,14.0,흥국화재,540.0,B+,C,B+,B+,2023.0,NaN,NaN,NaN,NaN
10986,4.0,흥아해운,3280.0,D,D,C,D,2020.0,NaN,NaN,NaN,NaN
10987,3.0,흥아해운,3280.0,C,C,B,C,2021.0,NaN,NaN,NaN,NaN
10988,2.0,흥아해운,3280.0,B,B,B+,B,2022.0,NaN,NaN,NaN,NaN


# 병합

In [12]:
merged_finance_esg = pd.merge(
    finance_data,
    filtered_data_year,
    left_on=['기업코드', '평가년도'],
    right_on=['기업코드', '평가년도'],
    how='inner'
)

In [13]:
merged_total = pd.merge(
    merged_finance_esg,
    marketcap_data,
    left_on=['기업코드', '평가년도'],
    right_on=['기업코드', '평가년도'],
    how='inner'
)

In [14]:
# 평가년도가 4개 이상인 기업코드만 필터링
merged_total = merged_total.groupby('기업코드').filter(lambda x: x['평가년도'].nunique() > 3)
print(len(merged_total))
print(merged_total['기업코드'].nunique())

3496
874


In [15]:
merged_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3496 entries, 0 to 3495
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0_x      3496 non-null   int64  
 1   Symbol            3496 non-null   object 
 2   Ticker_x          3496 non-null   object 
 3   Name              3496 non-null   object 
 4   종목명 (Full)        3496 non-null   object 
 5   기업명 (한글)          3496 non-null   object 
 6   거래소 업종            3484 non-null   object 
 7   거래소 업종코드          3484 non-null   object 
 8   결산월               3496 non-null   int64  
 9   회계년_x             3496 non-null   int64  
 10  주기                3496 non-null   object 
 11  자기자본비율(%)         3496 non-null   object 
 12  차입금의존도(%)         3496 non-null   float64
 13  총부채(천원)           3496 non-null   float64
 14  영업이익률(%)          3496 non-null   float64
 15  매출액(천원)           3496 non-null   float64
 16  총자본회전율(회)         3496 non-null   object 
 17  

# 병합된 Dataframe 전처리

In [16]:
merged_total = merged_total.drop(columns=['Unnamed: 0_x', 'Symbol', '주기', '기업코드', '평가년도', 'NO',
                                          '기업명', 'ESG 등급조정', '환경 등급조정', '사회 등급조정',
                                          '지배구조 등급조정','Unnamed: 0_y', 'Ticker_y', '회계년_y'])
merged_total = merged_total.rename(columns={'Ticker_x': 'Ticker', '회계년_x': '회계년'})

In [17]:
merged_total.columns

Index(['Ticker', 'Name', '종목명 (Full)', '기업명 (한글)', '거래소 업종', '거래소 업종코드', '결산월',
       '회계년', '자기자본비율(%)', '차입금의존도(%)', '총부채(천원)', '영업이익률(%)', '매출액(천원)',
       '총자본회전율(회)', '영업활동으로인한현금흐름(천원)', '당좌자산(천원)', '유동부채(천원)',
       'ROA(당기순이익)(%)', '신용등급 (채권, NICE)', '신용등급 (CP, NICE)', 'ESG등급', '환경',
       '사회', '지배구조', '시가총액(백만원)', 'WACC(시가총액평균)(%)'],
      dtype='object')

In [18]:
# Step 1: Remove rows with any NaN values in the selected columns
columns_to_remove = [
    '자기자본비율(%)', '차입금의존도(%)', '총부채(천원)', '영업이익률(%)', '매출액(천원)', 
    '총자본회전율(회)', '영업활동으로인한현금흐름(천원)', '당좌자산(천원)', '유동부채(천원)', 'ROA(당기순이익)(%)','WACC(시가총액평균)(%)'
]
cleaned_data = merged_total.dropna(subset=columns_to_remove)

In [19]:
valid_companies = cleaned_data.groupby('Ticker')['회계년'].nunique()
valid_companies = valid_companies[valid_companies > 3].index

# Filter the dataset to include only companies with more than 4 year of data
cleaned_data = cleaned_data[cleaned_data['Ticker'].isin(valid_companies)]

In [20]:
cleaned_data

,Ticker,Name,종목명 (Full),기업명 (한글),거래소 업종,거래소 업종코드,결산월,회계년,자기자본비율(%),차입금의존도(%),...,유동부채(천원),ROA(당기순이익)(%),"신용등급 (채권, NICE)","신용등급 (CP, NICE)",ESG등급,환경,사회,지배구조,시가총액(백만원),WACC(시가총액평균)(%)
0,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2020,79.82,5.57,...,4.441290e+10,7,NaN,NaN,B+,A,A+,B,544116848.0,7.18
1,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2021,76.94,3.90,...,5.306730e+10,12.88,NaN,NaN,A,A,A+,B+,526023507.0,8.25
2,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2022,80.52,1.23,...,4.608605e+10,9.94,NaN,NaN,A,A,A+,B+,371684753.0,9.39
3,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2023,75.72,9.69,...,4.177510e+10,9.12,NaN,NaN,B+,B+,A,B,519893771.0,8.52
4,A000660,SK하이닉스,에스케이하이닉스보통주,에스케이하이닉스(주),코스피 전기·전자,I.013,12,2020,77.37,12.78,...,7.488366e+09,6.9,AA,A1,A,B+,A+,A,86268280.0,7.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,A001770,SHD,SHD보통주,(주)신화다이나믹스,코스피 금속,I.011,12,2023,60.48,14.16,...,3.308929e+07,0.13,NaN,NaN,D,D,D,C,25695.0,5.38
3492,A008500,일정실업,일정실업보통주,일정실업(주),코스피 섬유·의류,I.006,12,2020,47.25,38.84,...,9.444573e+06,-31.84,NaN,NaN,C,C,C,C,22080.0,3.19
3493,A008500,일정실업,일정실업보통주,일정실업(주),코스피 섬유·의류,I.006,12,2021,38.49,44.24,...,1.757923e+07,-13.96,NaN,NaN,D,D,D,C,43680.0,4.18
3494,A008500,일정실업,일정실업보통주,일정실업(주),코스피 섬유·의류,I.006,12,2022,23.3,61.77,...,2.489083e+07,-16.55,NaN,NaN,D,D,D,C,19680.0,6.1


# 코스피 기업만 뽑기

In [21]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3244 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            3244 non-null   object 
 1   Name              3244 non-null   object 
 2   종목명 (Full)        3244 non-null   object 
 3   기업명 (한글)          3244 non-null   object 
 4   거래소 업종            3232 non-null   object 
 5   거래소 업종코드          3232 non-null   object 
 6   결산월               3244 non-null   int64  
 7   회계년               3244 non-null   int64  
 8   자기자본비율(%)         3244 non-null   object 
 9   차입금의존도(%)         3244 non-null   float64
 10  총부채(천원)           3244 non-null   float64
 11  영업이익률(%)          3244 non-null   float64
 12  매출액(천원)           3244 non-null   float64
 13  총자본회전율(회)         3244 non-null   object 
 14  영업활동으로인한현금흐름(천원)  3244 non-null   float64
 15  당좌자산(천원)          3244 non-null   float64
 16  유동부채(천원)          3244 non-null   float64
 17  

In [22]:
cleaned_data['Ticker'].nunique()

811

In [23]:
cleaned_data['Name'].value_counts().index

Index(['삼성전자', '선진', '유니켐', '신세계건설', '현대바이오랜드', '삼영', 'STX', '한국주철관', '유수홀딩스',
       '효성ITX',
       ...
       '삼아알미늄', '비에이치', '한세실업', '진에어', 'HJ중공업', '세진중공업', '현대홈쇼핑', '디아이', '드림텍',
       '일정실업'],
      dtype='object', name='Name', length=811)

In [24]:
cleaned_data.groupby('거래소 업종')['Name'].unique()

거래소 업종
코스닥 IT 서비스        [펄어비스, 위메이드, SOOP, 안랩, 컴투스, 웹젠, 골프존, KG이니시스, 사...
코스닥 건설                                    [LS마린솔루션, KCC건설, SGC E&C]
코스닥 금속                                                      [포스코엠텍]
코스닥 금융                                                    [NHN KCP]
코스닥 기계·장비         [이오테크닉스, 주성엔지니어링, 원익IPS, 피엔티, 유진테크, 에스에프에이, 고영...
코스닥 기타금융                                               [에코프로, 하림지주]
코스닥 농업·임업 및 어업                                              [농우바이오]
코스닥 비금속                                         [티씨케이, 삼표시멘트, 유진기업]
코스닥 오락·문화         [JYP Ent., 에스엠, CJ ENM, 스튜디오드래곤, 와이지엔터테인먼트, 드림...
코스닥 유통                                [다우데이타, CJ프레시웨이, 케이티알파, 코웰패션]
코스닥 음식료·담배                                               [하림, 매일유업]
코스닥 의료·정밀기기                                                  [클래시스]
코스닥 일반서비스         [알테오젠, 리가켐바이오, 메지온, 솔브레인홀딩스, 아난티, 메가스터디교육, 지씨셀...
코스닥 전기·전자         [에코프로비엠, 리노공업, 대주전자재료, 서진시스템, 덕산네오룩스, 에스앤에스텍, ...
코스닥 제약            [HLB, 삼천당제약, 휴젤, 파마리서치,

In [25]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3244 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            3244 non-null   object 
 1   Name              3244 non-null   object 
 2   종목명 (Full)        3244 non-null   object 
 3   기업명 (한글)          3244 non-null   object 
 4   거래소 업종            3232 non-null   object 
 5   거래소 업종코드          3232 non-null   object 
 6   결산월               3244 non-null   int64  
 7   회계년               3244 non-null   int64  
 8   자기자본비율(%)         3244 non-null   object 
 9   차입금의존도(%)         3244 non-null   float64
 10  총부채(천원)           3244 non-null   float64
 11  영업이익률(%)          3244 non-null   float64
 12  매출액(천원)           3244 non-null   float64
 13  총자본회전율(회)         3244 non-null   object 
 14  영업활동으로인한현금흐름(천원)  3244 non-null   float64
 15  당좌자산(천원)          3244 non-null   float64
 16  유동부채(천원)          3244 non-null   float64
 17  

In [26]:
cleaned_data.loc[cleaned_data['Name'] == '사조산업', '거래소 업종'] = '코스피 농업·임업 및 어업'
cleaned_data.loc[cleaned_data['Name'] == '신라교역', '거래소 업종'] = '코스피 농업·임업 및 어업'
cleaned_data.loc[cleaned_data['Name'] == '동원수산', '거래소 업종'] = '코스피 농업·임업 및 어업'
cleaned_data.loc[cleaned_data['Name'] == '사조산업', '거래소 업종코드'] = 'I.221'
cleaned_data.loc[cleaned_data['Name'] == '신라교역', '거래소 업종코드'] = 'I.221'
cleaned_data.loc[cleaned_data['Name'] == '동원수산', '거래소 업종코드'] = 'I.221'

In [27]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3244 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            3244 non-null   object 
 1   Name              3244 non-null   object 
 2   종목명 (Full)        3244 non-null   object 
 3   기업명 (한글)          3244 non-null   object 
 4   거래소 업종            3244 non-null   object 
 5   거래소 업종코드          3244 non-null   object 
 6   결산월               3244 non-null   int64  
 7   회계년               3244 non-null   int64  
 8   자기자본비율(%)         3244 non-null   object 
 9   차입금의존도(%)         3244 non-null   float64
 10  총부채(천원)           3244 non-null   float64
 11  영업이익률(%)          3244 non-null   float64
 12  매출액(천원)           3244 non-null   float64
 13  총자본회전율(회)         3244 non-null   object 
 14  영업활동으로인한현금흐름(천원)  3244 non-null   float64
 15  당좌자산(천원)          3244 non-null   float64
 16  유동부채(천원)          3244 non-null   float64
 17  

In [28]:
cleaned_data.groupby('거래소 업종')['Name'].unique().apply(len)

거래소 업종
코스닥 IT 서비스        12
코스닥 건설             3
코스닥 금속             1
코스닥 금융             1
코스닥 기계·장비         13
코스닥 기타금융           2
코스닥 농업·임업 및 어업     1
코스닥 비금속            3
코스닥 오락·문화          7
코스닥 유통             4
코스닥 음식료·담배         2
코스닥 의료·정밀기기        1
코스닥 일반서비스         16
코스닥 전기·전자         19
코스닥 제약            20
코스닥 종이·목재          1
코스닥 화학             9
코스피 IT 서비스        23
코스피 건설            25
코스피 금속            57
코스피 금융            52
코스피 기계·장비         30
코스피 농업·임업 및 어업     3
코스피 부동산            3
코스피 비금속           20
코스피 섬유·의류         26
코스피 오락·문화         13
코스피 운송·창고         23
코스피 운송장비·부품       50
코스피 유통            59
코스피 음식료·담배        37
코스피 의료·정밀기기        6
코스피 일반서비스         31
코스피 전기·가스          9
코스피 전기·전자         58
코스피 제약            45
코스피 제조             8
코스피 종이·목재         17
코스피 통신             4
코스피 화학            97
Name: Name, dtype: int64

In [29]:
df_kospi = cleaned_data[cleaned_data['거래소 업종'].str.startswith('코스피')]

In [30]:
df_kospi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2784 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2784 non-null   object 
 1   Name              2784 non-null   object 
 2   종목명 (Full)        2784 non-null   object 
 3   기업명 (한글)          2784 non-null   object 
 4   거래소 업종            2784 non-null   object 
 5   거래소 업종코드          2784 non-null   object 
 6   결산월               2784 non-null   int64  
 7   회계년               2784 non-null   int64  
 8   자기자본비율(%)         2784 non-null   object 
 9   차입금의존도(%)         2784 non-null   float64
 10  총부채(천원)           2784 non-null   float64
 11  영업이익률(%)          2784 non-null   float64
 12  매출액(천원)           2784 non-null   float64
 13  총자본회전율(회)         2784 non-null   object 
 14  영업활동으로인한현금흐름(천원)  2784 non-null   float64
 15  당좌자산(천원)          2784 non-null   float64
 16  유동부채(천원)          2784 non-null   float64
 17  

In [31]:
df_kospi['Ticker'].value_counts()

Ticker
A005930    4
A012800    4
A003780    4
A014280    4
A120030    4
          ..
A003920    4
A034120    4
A003280    4
A284740    4
A008500    4
Name: count, Length: 696, dtype: int64

In [32]:
df_kospi.groupby('거래소 업종')['Name'].unique().apply(len)

거래소 업종
코스피 IT 서비스        23
코스피 건설            25
코스피 금속            57
코스피 금융            52
코스피 기계·장비         30
코스피 농업·임업 및 어업     3
코스피 부동산            3
코스피 비금속           20
코스피 섬유·의류         26
코스피 오락·문화         13
코스피 운송·창고         23
코스피 운송장비·부품       50
코스피 유통            59
코스피 음식료·담배        37
코스피 의료·정밀기기        6
코스피 일반서비스         31
코스피 전기·가스          9
코스피 전기·전자         58
코스피 제약            45
코스피 제조             8
코스피 종이·목재         17
코스피 통신             4
코스피 화학            97
Name: Name, dtype: int64

In [33]:
df_kospi.columns

Index(['Ticker', 'Name', '종목명 (Full)', '기업명 (한글)', '거래소 업종', '거래소 업종코드', '결산월',
       '회계년', '자기자본비율(%)', '차입금의존도(%)', '총부채(천원)', '영업이익률(%)', '매출액(천원)',
       '총자본회전율(회)', '영업활동으로인한현금흐름(천원)', '당좌자산(천원)', '유동부채(천원)',
       'ROA(당기순이익)(%)', '신용등급 (채권, NICE)', '신용등급 (CP, NICE)', 'ESG등급', '환경',
       '사회', '지배구조', '시가총액(백만원)', 'WACC(시가총액평균)(%)'],
      dtype='object')

# 추가 전처리들(단위 변경 등)

In [34]:
df_copy = df_kospi.copy()

In [35]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2784 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2784 non-null   object 
 1   Name              2784 non-null   object 
 2   종목명 (Full)        2784 non-null   object 
 3   기업명 (한글)          2784 non-null   object 
 4   거래소 업종            2784 non-null   object 
 5   거래소 업종코드          2784 non-null   object 
 6   결산월               2784 non-null   int64  
 7   회계년               2784 non-null   int64  
 8   자기자본비율(%)         2784 non-null   object 
 9   차입금의존도(%)         2784 non-null   float64
 10  총부채(천원)           2784 non-null   float64
 11  영업이익률(%)          2784 non-null   float64
 12  매출액(천원)           2784 non-null   float64
 13  총자본회전율(회)         2784 non-null   object 
 14  영업활동으로인한현금흐름(천원)  2784 non-null   float64
 15  당좌자산(천원)          2784 non-null   float64
 16  유동부채(천원)          2784 non-null   float64
 17  

In [36]:
# "총자본회전율(회)"이 완전잠식인 회사 2개 존재
non_numeric_values = df_copy[~df_copy['총자본회전율(회)'].astype(str).str.replace('.', '', 1).str.isdigit()]
print(non_numeric_values[['총자본회전율(회)']])

     총자본회전율(회)
1304      완전잠식


In [37]:
# "총자본회전율(회)"이 완전잠식인 행들 제외
df_copy = df_copy[df_copy["총자본회전율(회)"] != "완전잠식"] 
df_copy["총자본회전율(회)"] = pd.to_numeric(df_copy["총자본회전율(회)"], errors='coerce')

# Ticker 등장 횟수 계산
ticker_counts = df_copy['Ticker'].value_counts()

# 4개 이상 등장한 Ticker 목록 추출
tickers_to_keep = ticker_counts[ticker_counts >= 4].index

# df_copy에서 4개 이상 등장한 Ticker만 선택
df_copy = df_copy[df_copy['Ticker'].isin(tickers_to_keep)]

In [38]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2780 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2780 non-null   object 
 1   Name              2780 non-null   object 
 2   종목명 (Full)        2780 non-null   object 
 3   기업명 (한글)          2780 non-null   object 
 4   거래소 업종            2780 non-null   object 
 5   거래소 업종코드          2780 non-null   object 
 6   결산월               2780 non-null   int64  
 7   회계년               2780 non-null   int64  
 8   자기자본비율(%)         2780 non-null   object 
 9   차입금의존도(%)         2780 non-null   float64
 10  총부채(천원)           2780 non-null   float64
 11  영업이익률(%)          2780 non-null   float64
 12  매출액(천원)           2780 non-null   float64
 13  총자본회전율(회)         2780 non-null   float64
 14  영업활동으로인한현금흐름(천원)  2780 non-null   float64
 15  당좌자산(천원)          2780 non-null   float64
 16  유동부채(천원)          2780 non-null   float64
 17  

In [39]:
# "자기자본비율(%)"이 완전잠식인 회사 4개 존재(했음)
non_numeric_values = df_copy[~df_copy["자기자본비율(%)"].astype(str).str.replace('.', '', 1).str.isdigit()]
print(non_numeric_values[["자기자본비율(%)"]])

     자기자본비율(%)
1023      완전잠식
1095      완전잠식


In [40]:
# "자기자본비율(%)"이 완전잠식인 행들 제외
df_copy = df_copy[df_copy["자기자본비율(%)"] != "완전잠식"] 
df_copy["자기자본비율(%)"] = pd.to_numeric(df_copy["자기자본비율(%)"], errors='coerce')

# Ticker 등장 횟수 계산
ticker_counts = df_copy['Ticker'].value_counts()

# 4개 이상 등장한 Ticker 목록 추출
tickers_to_keep = ticker_counts[ticker_counts >= 4].index

# df_copy에서 4개 이상 등장한 Ticker만 선택
df_copy = df_copy[df_copy['Ticker'].isin(tickers_to_keep)]

In [41]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2772 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2772 non-null   object 
 1   Name              2772 non-null   object 
 2   종목명 (Full)        2772 non-null   object 
 3   기업명 (한글)          2772 non-null   object 
 4   거래소 업종            2772 non-null   object 
 5   거래소 업종코드          2772 non-null   object 
 6   결산월               2772 non-null   int64  
 7   회계년               2772 non-null   int64  
 8   자기자본비율(%)         2772 non-null   float64
 9   차입금의존도(%)         2772 non-null   float64
 10  총부채(천원)           2772 non-null   float64
 11  영업이익률(%)          2772 non-null   float64
 12  매출액(천원)           2772 non-null   float64
 13  총자본회전율(회)         2772 non-null   float64
 14  영업활동으로인한현금흐름(천원)  2772 non-null   float64
 15  당좌자산(천원)          2772 non-null   float64
 16  유동부채(천원)          2772 non-null   float64
 17  

In [42]:
df_copy["자기자본비율(%)"].describe()

count    2772.00000
mean       61.05009
std        20.80279
min         1.46000
25%        46.73500
50%        60.50500
75%        78.81000
max        99.93000
Name: 자기자본비율(%), dtype: float64

In [43]:
# 수치형 변수 단위 통일
df_copy["시가총액(백만원)"] = df_copy["시가총액(백만원)"] * 1000
df_copy = df_copy.rename(columns={'시가총액(백만원)':'시가총액(천원)'})

In [44]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2772 entries, 0 to 3495
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2772 non-null   object 
 1   Name              2772 non-null   object 
 2   종목명 (Full)        2772 non-null   object 
 3   기업명 (한글)          2772 non-null   object 
 4   거래소 업종            2772 non-null   object 
 5   거래소 업종코드          2772 non-null   object 
 6   결산월               2772 non-null   int64  
 7   회계년               2772 non-null   int64  
 8   자기자본비율(%)         2772 non-null   float64
 9   차입금의존도(%)         2772 non-null   float64
 10  총부채(천원)           2772 non-null   float64
 11  영업이익률(%)          2772 non-null   float64
 12  매출액(천원)           2772 non-null   float64
 13  총자본회전율(회)         2772 non-null   float64
 14  영업활동으로인한현금흐름(천원)  2772 non-null   float64
 15  당좌자산(천원)          2772 non-null   float64
 16  유동부채(천원)          2772 non-null   float64
 17  

In [45]:
df_copy['Ticker'].value_counts()

Ticker
A005930    4
A012610    4
A120030    4
A058850    4
A015590    4
          ..
A034120    4
A284740    4
A195870    4
A003300    4
A008500    4
Name: count, Length: 693, dtype: int64

# 매출액 -> 매출액 증가율로 변경

In [46]:
finance_file_path = 'finance.csv'
finance_data = pd.read_csv(finance_file_path, encoding="cp949", low_memory=False)

In [47]:
finance_2019 = finance_data[(finance_data['회계년'] == 2019) & (finance_data['Ticker'].isin(df_copy['Ticker']))]
finance_2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 693 entries, 7 to 31387
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        693 non-null    int64  
 1   Symbol            693 non-null    object 
 2   Ticker            693 non-null    object 
 3   Name              693 non-null    object 
 4   종목명 (Full)        693 non-null    object 
 5   기업명 (한글)          693 non-null    object 
 6   거래소 업종            690 non-null    object 
 7   거래소 업종코드          690 non-null    object 
 8   결산월               693 non-null    int64  
 9   회계년               693 non-null    int64  
 10  주기                693 non-null    object 
 11  자기자본비율(%)         693 non-null    object 
 12  차입금의존도(%)         693 non-null    float64
 13  총부채(천원)           693 non-null    float64
 14  영업이익률(%)          693 non-null    float64
 15  매출액(천원)           693 non-null    float64
 16  총자본회전율(회)         691 non-null    object 
 17  

In [48]:
np.sum(finance_2019['매출액(천원)'].isnull())

0

In [49]:
df_updated = pd.concat([df_copy, finance_2019])

In [50]:
df_updated.info()
# 684 * 5 (19~23년도도)

<class 'pandas.core.frame.DataFrame'>
Index: 3465 entries, 0 to 31387
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            3465 non-null   object 
 1   Name              3465 non-null   object 
 2   종목명 (Full)        3465 non-null   object 
 3   기업명 (한글)          3465 non-null   object 
 4   거래소 업종            3462 non-null   object 
 5   거래소 업종코드          3462 non-null   object 
 6   결산월               3465 non-null   int64  
 7   회계년               3465 non-null   int64  
 8   자기자본비율(%)         3465 non-null   object 
 9   차입금의존도(%)         3465 non-null   float64
 10  총부채(천원)           3465 non-null   float64
 11  영업이익률(%)          3465 non-null   float64
 12  매출액(천원)           3465 non-null   float64
 13  총자본회전율(회)         3463 non-null   object 
 14  영업활동으로인한현금흐름(천원)  3465 non-null   float64
 15  당좌자산(천원)          3465 non-null   float64
 16  유동부채(천원)          3465 non-null   float64
 17 

In [51]:
df_updated = df_updated.drop(columns=['Symbol', '주기'])

In [52]:
df_updated.head(3)

,Ticker,Name,종목명 (Full),기업명 (한글),거래소 업종,거래소 업종코드,결산월,회계년,자기자본비율(%),차입금의존도(%),...,ROA(당기순이익)(%),"신용등급 (채권, NICE)","신용등급 (CP, NICE)",ESG등급,환경,사회,지배구조,시가총액(천원),WACC(시가총액평균)(%),Unnamed: 0
0,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2020,79.82,5.57,...,7,NaN,NaN,B+,A,A+,B,5.441168e+11,7.18,NaN
1,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2021,76.94,3.90,...,12.88,NaN,NaN,A,A,A+,B+,5.260235e+11,8.25,NaN
2,A005930,삼성전자,삼성전자보통주,삼성전자(주),코스피 전기·전자,I.013,12,2022,80.52,1.23,...,9.94,NaN,NaN,A,A,A+,B+,3.716848e+11,9.39,NaN


In [53]:
df = df_updated.sort_values(by=['Ticker', '회계년'])

In [54]:
df['Ticker'].value_counts()

Ticker
A000020    5
A034310    5
A033530    5
A033780    5
A033920    5
          ..
A007120    5
A007160    5
A007210    5
A007280    5
A336370    5
Name: count, Length: 693, dtype: int64

In [55]:
df['회계년'].value_counts()

회계년
2019    693
2020    693
2021    693
2022    693
2023    693
Name: count, dtype: int64

In [56]:
# 이전 년도의 매출액을 새로운 컬럼 '이전년도매출액(천원)'에 저장
df['이전년도매출액(천원)'] = df.groupby('Ticker')['매출액(천원)'].shift(1) 

# 매출액증가율 계산
df['매출액증가율(%)'] = (df['매출액(천원)'] / df['이전년도매출액(천원)']) - 1
df['매출액증가율(%)'] = df['매출액증가율(%)'] * 100.

In [57]:
df_high = df.copy()

In [58]:
df_high['매출액증가율(%)'].describe()

count     2772.000000
mean        22.971290
std        541.365092
min        -99.967293
25%         -6.606205
50%          5.142367
75%         18.165214
max      28139.380905
Name: 매출액증가율(%), dtype: float64

In [59]:
df_high['매출액(천원)'].describe()

count    3.465000e+03
mean     1.835560e+09
std      8.631913e+09
min      3.762813e+05
25%      1.085716e+08
50%      2.904975e+08
75%      9.263986e+08
max      2.118670e+11
Name: 매출액(천원), dtype: float64

In [60]:
df_2020 = df_high[df_high['회계년'] != 2019]

In [61]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2772 entries, 2060 to 1059
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2772 non-null   object 
 1   Name              2772 non-null   object 
 2   종목명 (Full)        2772 non-null   object 
 3   기업명 (한글)          2772 non-null   object 
 4   거래소 업종            2772 non-null   object 
 5   거래소 업종코드          2772 non-null   object 
 6   결산월               2772 non-null   int64  
 7   회계년               2772 non-null   int64  
 8   자기자본비율(%)         2772 non-null   object 
 9   차입금의존도(%)         2772 non-null   float64
 10  총부채(천원)           2772 non-null   float64
 11  영업이익률(%)          2772 non-null   float64
 12  매출액(천원)           2772 non-null   float64
 13  총자본회전율(회)         2772 non-null   object 
 14  영업활동으로인한현금흐름(천원)  2772 non-null   float64
 15  당좌자산(천원)          2772 non-null   float64
 16  유동부채(천원)          2772 non-null   float64
 1

In [62]:
df_2020['부채 상환능력(%)'] = (df_2020['영업활동으로인한현금흐름(천원)'] / df_2020['총부채(천원)']) * 100

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\1508299387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['부채 상환능력(%)'] = (df_2020['영업활동으로인한현금흐름(천원)'] / df_2020['총부채(천원)']) * 100


In [63]:
df_2020['부채 상환능력(%)'].describe()

count    2772.000000
mean       26.022223
std       187.840756
min      -399.546902
25%         1.310985
50%        11.528111
75%        26.427288
max      8270.425918
Name: 부채 상환능력(%), dtype: float64

In [64]:
df_2020['당좌비율(%)'] = (df_2020['당좌자산(천원)'] / df_2020['유동부채(천원)']) * 100

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\1549935075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['당좌비율(%)'] = (df_2020['당좌자산(천원)'] / df_2020['유동부채(천원)']) * 100


In [65]:
# 선택할 컬럼 리스트 (제시한 순서대로 정렬)
selected_columns = [
    'Ticker', 'Name', '종목명 (Full)', '기업명 (한글)', '거래소 업종', '거래소 업종코드',
    '결산월', '회계년', '자기자본비율(%)', '차입금의존도(%)', '매출액증가율(%)', '영업이익률(%)', 
    '총자본회전율(회)', '부채 상환능력(%)', '당좌비율(%)', 'ROA(당기순이익)(%)', 
    '신용등급 (채권, NICE)', '신용등급 (CP, NICE)', 'ESG등급', '환경', '사회', '지배구조', 
    '시가총액(천원)', 'WACC(시가총액평균)(%)'
]

# 새로운 데이터프레임 생성 (컬럼 순서 적용)
df_final = df_2020[selected_columns]

In [66]:
df_2020["자기자본비율(%)"] = pd.to_numeric(df_2020["자기자본비율(%)"], errors='coerce')
df_2020["총자본회전율(회)"] = pd.to_numeric(df_2020["총자본회전율(회)"], errors='coerce')
df_2020["ROA(당기순이익)(%)"] = pd.to_numeric(df_2020["ROA(당기순이익)(%)"], errors='coerce')
df_2020["WACC(시가총액평균)(%)"] = pd.to_numeric(df_2020["WACC(시가총액평균)(%)"], errors='coerce')

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\2362784477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["자기자본비율(%)"] = pd.to_numeric(df_2020["자기자본비율(%)"], errors='coerce')
C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\2362784477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["총자본회전율(회)"] = pd.to_numeric(df_2020["총자본회전율(회)"], errors='coerce')
C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\2362784477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [67]:
df_2020["총자본회전율(회)"] = df_2020["총자본회전율(회)"] * 100
df_2020 = df_2020.rename(columns={'총자본회전율(회)':'총자본회전율(%)'})

C:\Users\jwlee\AppData\Local\Temp\ipykernel_15924\1061948644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020["총자본회전율(회)"] = df_2020["총자본회전율(회)"] * 100


In [68]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2772 entries, 2060 to 1059
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Ticker            2772 non-null   object 
 1   Name              2772 non-null   object 
 2   종목명 (Full)        2772 non-null   object 
 3   기업명 (한글)          2772 non-null   object 
 4   거래소 업종            2772 non-null   object 
 5   거래소 업종코드          2772 non-null   object 
 6   결산월               2772 non-null   int64  
 7   회계년               2772 non-null   int64  
 8   자기자본비율(%)         2772 non-null   float64
 9   차입금의존도(%)         2772 non-null   float64
 10  총부채(천원)           2772 non-null   float64
 11  영업이익률(%)          2772 non-null   float64
 12  매출액(천원)           2772 non-null   float64
 13  총자본회전율(%)         2772 non-null   float64
 14  영업활동으로인한현금흐름(천원)  2772 non-null   float64
 15  당좌자산(천원)          2772 non-null   float64
 16  유동부채(천원)          2772 non-null   float64
 1

In [69]:
df_2020.head()

,Ticker,Name,종목명 (Full),기업명 (한글),거래소 업종,거래소 업종코드,결산월,회계년,자기자본비율(%),차입금의존도(%),...,환경,사회,지배구조,시가총액(천원),WACC(시가총액평균)(%),Unnamed: 0,이전년도매출액(천원),매출액증가율(%),부채 상환능력(%),당좌비율(%)
2060,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2020,81.70,1.40,...,D,B+,B,548853000.0,5.54,NaN,307150025.8,-12.928899,68.804780,315.312431
2061,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2021,82.70,0.78,...,C,B+,C,407799000.0,5.12,NaN,267438909.2,1.785185,42.403687,289.155054
2062,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2022,83.70,0.78,...,C,B,C,258645000.0,10.28,NaN,272213188.3,16.059010,38.869866,239.866410
2063,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2023,79.27,4.35,...,B,B,C,286856000.0,6.33,NaN,315927932.0,6.079589,28.783839,166.229629
3412,A000040,KR모터스,KR모터스보통주,케이알모터스(주),코스피 운송장비·부품,I.015,12,2020,38.40,24.45,...,D,C,B,104035000.0,8.58,NaN,37365835.5,-45.891737,-10.544052,14.391917


In [70]:
# 선택할 컬럼 리스트 (제시한 순서대로 정렬)
selected_columns = [
    'Ticker', 'Name', '종목명 (Full)', '기업명 (한글)', '거래소 업종', '거래소 업종코드',
    '결산월', '회계년', '자기자본비율(%)', '차입금의존도(%)', '매출액증가율(%)', '영업이익률(%)', 
    '총자본회전율(%)', '부채 상환능력(%)', '당좌비율(%)', 'ROA(당기순이익)(%)', 
    '신용등급 (채권, NICE)', '신용등급 (CP, NICE)', 'ESG등급', '환경', '사회', '지배구조', 
    '시가총액(천원)', 'WACC(시가총액평균)(%)'
]

# 새로운 데이터프레임 생성 (컬럼 순서 적용)
df_final = df_2020[selected_columns]

In [71]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2772 entries, 2060 to 1059
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ticker           2772 non-null   object 
 1   Name             2772 non-null   object 
 2   종목명 (Full)       2772 non-null   object 
 3   기업명 (한글)         2772 non-null   object 
 4   거래소 업종           2772 non-null   object 
 5   거래소 업종코드         2772 non-null   object 
 6   결산월              2772 non-null   int64  
 7   회계년              2772 non-null   int64  
 8   자기자본비율(%)        2772 non-null   float64
 9   차입금의존도(%)        2772 non-null   float64
 10  매출액증가율(%)        2772 non-null   float64
 11  영업이익률(%)         2772 non-null   float64
 12  총자본회전율(%)        2772 non-null   float64
 13  부채 상환능력(%)       2772 non-null   float64
 14  당좌비율(%)          2772 non-null   float64
 15  ROA(당기순이익)(%)    2772 non-null   float64
 16  신용등급 (채권, NICE)  710 non-null    object 
 17  신용등급 (CP, NICE) 

In [72]:
df_final.head()

,Ticker,Name,종목명 (Full),기업명 (한글),거래소 업종,거래소 업종코드,결산월,회계년,자기자본비율(%),차입금의존도(%),...,당좌비율(%),ROA(당기순이익)(%),"신용등급 (채권, NICE)","신용등급 (CP, NICE)",ESG등급,환경,사회,지배구조,시가총액(천원),WACC(시가총액평균)(%)
2060,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2020,81.70,1.40,...,315.312431,7.27,NaN,NaN,B,D,B+,B,548853000.0,5.54
2061,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2021,82.70,0.78,...,289.155054,3.59,NaN,NaN,C,C,B+,C,407799000.0,5.12
2062,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2022,83.70,0.78,...,239.866410,4.30,NaN,NaN,C,C,B,C,258645000.0,10.28
2063,A000020,동화약품,동화약품보통주,동화약품(주),코스피 제약,I.009,12,2023,79.27,4.35,...,166.229629,5.79,NaN,NaN,C,B,B,C,286856000.0,6.33
3412,A000040,KR모터스,KR모터스보통주,케이알모터스(주),코스피 운송장비·부품,I.015,12,2020,38.40,24.45,...,14.391917,-11.10,B-,NaN,C,D,C,B,104035000.0,8.58


In [73]:
df_final.to_csv('preprocess250201.csv')